In [83]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import numpy as np


In [84]:
imported_to='2023-03-13'
# Get Last Upldate from BQ update data
dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

removeCols=dateCols+['open_to_close','response_to_resolved']

numbericCols=['open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type']

file_name="New_Incident.csv"
table_id = "pongthorn.SMartML.new_incident"


In [85]:
sql=f"""
SELECT  id,
severity_id,severity_name,sla,
product_type,brand,service_type,incident_type,
open_datetime,  close_datetime, response_datetime,resolved_datetime, imported_at
FROM `pongthorn.SMartDW.incident` WHERE imported_at>= '{imported_to}'
order by imported_at
"""

client_bq = bigquery.Client()
query_result=client_bq.query(sql)
df_all=query_result.to_dataframe()
df_all.head()

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_datetime,close_datetime,response_datetime,resolved_datetime,imported_at
0,1867,4,Cosmatic,24x7 4Hrs Resolution Time,Other,F5,Request,Configuration Change,2022-11-18 15:17:00,2023-03-13 12:15:00,2022-11-18 15:21:00,2023-01-23 13:48:00,2023-03-13 21:00:05.778139
1,2428,3,Minor,24x7 6Hrs Resolution Time,Software,SAPB1,Incident,Software,2023-03-07 14:58:00,2023-03-10 17:00:00,2023-03-07 15:00:00,2023-03-10 15:05:00,2023-03-13 21:00:05.778139
2,2420,2,Major,24x7 4Hrs Resolution Time,Software,VMWare,Incident,Software,2023-03-09 04:46:00,2023-03-09 06:10:00,2023-03-09 05:07:00,2023-03-09 06:10:00,2023-03-13 21:00:05.778139
3,2410,3,Minor,24x7 4Hrs Response Time,Software,Trend Micro,Request,Software,2023-03-08 11:34:00,2023-03-08 13:34:00,2023-03-08 11:39:00,2023-03-08 13:34:00,2023-03-13 21:00:05.778139
4,2342,2,Major,24x7 4Hrs Resolution Time,Software,Veeam,Incident,Software,2023-02-19 21:24:00,2023-03-13 17:35:00,2023-02-19 21:32:00,2023-03-13 17:35:00,2023-03-13 21:00:05.778139


In [86]:
start_end_list=[ ['open_datetime','close_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


In [87]:
print(f"Max Last Import ={df_all['imported_at'].max()}")

for col in numbericCols:
 df_all=df_all.query(f'{col}!=0')

df_all=df_all.drop_duplicates(subset=['id'],keep='first')
df_all=df_all.drop(columns=removeCols)

print(df_all.info())
df_all.tail()

Max Last Import =2023-03-20 21:00:07.377573
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 53
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         50 non-null     Int64         
 1   severity_id                50 non-null     Int64         
 2   severity_name              50 non-null     object        
 3   sla                        50 non-null     object        
 4   product_type               50 non-null     object        
 5   brand                      50 non-null     object        
 6   service_type               50 non-null     object        
 7   incident_type              50 non-null     object        
 8   imported_at                50 non-null     datetime64[ns]
 9   open_to_close_hour         50 non-null     float64       
 10  response_to_resolved_hour  50 non-null     float64       
dtypes: Int64(2), datetime64[ns](1

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,imported_at,open_to_close_hour,response_to_resolved_hour
49,2468,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,13.016667,0.866667
50,2451,2,Major,24x7 4Hrs Response Time,Software,Veritas,Incident,Software,2023-03-20 21:00:07.377573,13.000000,12.750000
51,2470,4,Cosmatic,24x7 4Hrs Response Time,Software,Red Hat,Request,General Incident,2023-03-20 21:00:07.377573,102.650000,98.016667
52,2437,4,Cosmatic,24x7 4Hrs Resolution Time,Software,Red Hat,Request,Software,2023-03-20 21:00:07.377573,47.366667,23.016667
53,2279,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,2023-03-20 21:00:07.377573,868.283333,868.133333


In [89]:
#df_all[numbericCols].describe()

,open_to_close_hour,response_to_resolved_hour
count,50.000000,50.000000
mean,441.403000,410.288000
std,1896.686838,1875.715138
min,0.083333,0.066667
25%,4.175000,1.266667
50%,17.883333,12.000000
75%,93.270833,72.062500
max,13177.250000,13176.250000


In [90]:
client = bigquery.Client()
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df_all, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_all), "Imported igquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex


Total  50 Imported igquery successfully
